In [3]:
!python --version

Python 3.11.9


In [1]:
import contextlib
from flask import Flask, jsonify, request, g
import requests
import asyncio
import websockets
import json
from rtsp_videocapture import VideoCapture
import time
import numpy as np
import tensorflow as tf
import os, cv2
import threading

app = Flask(__name__)


@app.route('/')
def home():
    return "Welcome to the Flask app!"

@app.route('/send', methods=['POST'])
def send_message():

    message = {
        "name": "John",
        "message": "Hello, World!"
    }

    headers = {'Content-Type': 'application/json'}
    response = requests.post("http://127.0.0.1:5000/receive", json=message, headers=headers)

    return jsonify({"status": "Message sent", "response": response.json()})

@app.route('/data', methods=['POST'])
def receive_message():

    data = request.get_json()
    print("\ndata['command']: ", data['command'])


    if data['command'] == "forward": # Send donkeycar Web (Run Car)
        print('one !!!')
        
        ### 第一次: Java Servlet 叫車子走去 Sender 部門 ###
        if 'recipientDepartment' in data:
            with open('recipient_dept.txt', 'w') as file:

                # 把網頁後端這次Post中的 Recipient Dept 先寫到 txt 保存起來
                # 等著 Sender 放好物品、按下按鈕後
                # 要用來餵給Model時使用 (比對 辨識到收件部門的動物圖片)
                # 因為當這次POST結束後 data 變數的內容 將會消失
                file.write(data['recipientDepartment'])

            # 從 dept2cat.json 讀取 部門代號 轉 動物品種
            # Json = Dictionary = Hashmap
            file_path = 'dept2cat.json'
            with open(file_path, 'r') as file:
                dept2cat = json.load(file)

            # 在Python裡把 部門代號(A/B/C/D) 轉成 動物品種
            recipient_dept = data['senderDepartment'] # data['senderDepartment'] 的值是 A/B/C/D
            target_animal = dept2cat[recipient_dept] # 假設: 部門代號A 會拿到 'tiger'

            # 開1個Thread 連讀 RTSP 相機影像，辨識 dept2cat[recipient_dept] 拿到的動物品種
            threadrtsp = threading.Thread(target=ai_image_detect, args=(target_animal,))
            threadrtsp.start()


        ### 第二次: Sender 放好物品，按下按鈕 叫車子走去 Recipient 部門 ###
        elif 'donkeycar' in data:

            file_path = 'dept2cat.json'
            with open(file_path, 'r') as file:
                dept2cat = json.load(file)


            # 這次的送件部門 是用讀取 recipient_dept.txt 的
            with open('recipient_dept.txt', 'r') as file:
                recipient_dept = file.read()

            target_animal = dept2cat[recipient_dept] # C -> 'leopard'

            threadrtsp = threading.Thread(target=ai_image_detect, args=(target_animal,))
            threadrtsp.start()


        # time.sleep(1)
        asyncio.run(send_data('local'))

    
    elif data['command'] == "stop": # Send donkeycar Web (Stop Car)
        print('stop')
        asyncio.run(send_data('user'))

    else:
        raise Exception(f"Get wrong command: {data['command']}")
    return jsonify({"status": "Message received", "data": data})

def ai_image_detect(target_label):
    print('\n\ntarget_label: ', target_label)

    vs = VideoCapture("rtsp://192.168.93.132:8554/unicast")
    print('rtsp')
    start_time2 = time.time()
    frame_rate = 35
    fps = 0
    
    model = tf.keras.applications.InceptionV3(include_top=True, weights='imagenet')
    imagenet_labels = read_label_list()

    
    while True:
        frame,success = vs.read()
        start_time = time.time()
        if not success:
            break

        resized_frame = frame.copy()
        resized_frame = cv2.resize(resized_frame, (299, 299))
        frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
        frame_tensor = tf.convert_to_tensor(frame_rgb, dtype=tf.float32)
        frame_tensor = frame_tensor / 255.0
        frame_tensor= tf.expand_dims(frame_tensor, 0)
        
        with contextlib.redirect_stdout(open(os.devnull, 'w')):
            preds = model.predict(frame_tensor)
        index = np.argmax(preds)

        predict_result = imagenet_labels[index+1]
        # print("Predicted:", predict_result)

        if predict_result == target_label: # Detect target dept
            asyncio.run(send_data('user'))  # !!! Stop !!!
            break


        loop_time = time.time() - start_time
        delay = max(1, int((1 / frame_rate - loop_time) * 1000))
        key = cv2.waitKey(delay) & 0xFF

        if key == ord('q'):
            break

        loop_time2 = time.time() - start_time
        if loop_time2 > 0:
            fps = 0.9 * fps + 0.1 / loop_time2
            # print(fps)

        cv2.putText(frame, f"FPS: {fps:.2f} / {predict_result}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.imshow("Live Stream", frame)


    total_time = time.time() - start_time2
    print("Total time taken:", total_time, "seconds")
    
    cv2.destroyAllWindows()
    vs.cap.release()

def read_label_list():
    # current_script_path = os.path.abspath(__file__)
    # current_script_directory = os.path.dirname(current_script_path)

    # print(current_script_directory)
    labels_path = 'C:\\Users\\Gaming\\Desktop\\AILM - iSpan\\專題\\AIML05_FinalProject\\robot_service\\ImageNetLabels.txt'

    with open(labels_path) as file:
        lines = file.read().splitlines()
    print(lines)

    return np.array(lines)

async def send_data(drive_mode):
    data = {}
    fields = ['angle', 'throttle', 'drive_mode', 'recording', 'buttons']  # Assuming 'fields' array in JavaScript context
    for field in fields:
        if field == 'angle':
            data['angle'] = 0
        elif field == 'throttle':
            data['throttle'] = 0
        elif field == 'drive_mode':
            data['drive_mode'] = drive_mode
        elif field == 'recording':
            data['recording'] = False
        elif field == 'buttons':
            data['buttons'] = {"w1":False,"w2":False,"w3":False,"w4":False,"w5":False}
        else:
            print(f"Unexpected post field: '{field}'")

    if data:
        json_data = json.dumps(data)
        print(f"Posting {json_data}")
        async with websockets.connect('ws://192.168.93.234:8887/wsDrive') as websocket:
            # print('ok')
            await websocket.send(json_data)


if __name__ == "__main__":
    app.run(host='0.0.0.0', port = 5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.93.31:5000
Press CTRL+C to quit
192.168.93.51 - - [21/Jun/2024 15:31:02] "POST /data HTTP/1.1" 200 -
192.168.93.51 - - [21/Jun/2024 15:31:02] "POST /data HTTP/1.1" 200 -
192.168.93.51 - - [21/Jun/2024 15:31:02] "POST /data HTTP/1.1" 200 -



data['command']: 
data['command']:  stop
stop
 stop
stop

data['command']:  stop
stop
Posting {"angle": 0, "throttle": 0, "drive_mode": "user", "recording": false, "buttons": {"w1": false, "w2": false, "w3": false, "w4": false, "w5": false}}
Posting {"angle": 0, "throttle": 0, "drive_mode": "user", "recording": false, "buttons": {"w1": false, "w2": false, "w3": false, "w4": false, "w5": false}}
Posting {"angle": 0, "throttle": 0, "drive_mode": "user", "recording": false, "buttons": {"w1": false, "w2": false, "w3": false, "w4": false, "w5": false}}


192.168.93.51 - - [21/Jun/2024 15:31:03] "POST /data HTTP/1.1" 200 -
192.168.93.51 - - [21/Jun/2024 15:31:03] "POST /data HTTP/1.1" 200 -
192.168.93.51 - - [21/Jun/2024 15:31:03] "POST /data HTTP/1.1" 200 -



data['command']:  forward
one !!!


target_label:  tiger
Posting {"angle": 0, "throttle": 0, "drive_mode": "local", "recording": false, "buttons": {"w1": false, "w2": false, "w3": false, "w4": false, "w5": false}}

data['command']:  forward
one !!!


target_label:  tiger
Posting {"angle": 0, "throttle": 0, "drive_mode": "local", "recording": false, "buttons": {"w1": false, "w2": false, "w3": false, "w4": false, "w5": false}}

data['command']:  forward
one !!!


target_label:  tiger
Posting {"angle": 0, "throttle": 0, "drive_mode": "local", "recording": false, "buttons": {"w1": false, "w2": false, "w3": false, "w4": false, "w5": false}}
rtsp
rtsp
['background', 'tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead', 'electric ray', 'stingray', 'cock', 'hen', 'ostrich', 'brambling', 'goldfinch', 'house finch', 'junco', 'indigo bunting', 'robin', 'bulbul', 'jay', 'magpie', 'chickadee', 'water ouzel', 'kite', 'bald eagle', 'vulture', 'great grey owl', 'European fire salaman

192.168.93.51 - - [21/Jun/2024 15:32:05] "POST /data HTTP/1.1" 200 -
Exception in thread Thread-14 (_reader):
Traceback (most recent call last):
  File "c:\Users\Gaming\anaconda3\envs\py311_delivery_robot\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\Gaming\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Gaming\anaconda3\envs\py311_delivery_robot\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Servlet\workspace\AIML05_FinalProject\robot_service\rtsp_videocapture.py", line 23, in _reader
    ret, frame = self.cap.read()
                 ^^^^^^^^^^^^^^^
cv2.error: Unknown C++ exception from OpenCV code
Exception in thread Thread-15 (_reader):
Traceback (most recent call last):
  File "c:\Users\Gaming\anaconda3\envs\py311_delivery_robot\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Us